In [126]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import re

from bs4 import BeautifulSoup

In [127]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url, headers=headers )


In [128]:
soup = BeautifulSoup( page.text, 'html.parser' )

In [129]:
products = soup.find( 'ul', class_='products-listing small' )

product_list = products.find_all( 'article', class_='hm-product-item')
# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]
# product category
product_category = [p.get( 'data-category' ) for p in product_list]

## product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]

In [130]:
#create the DataFrame
data = pd.DataFrame( [product_id, product_category, product_name,product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name','product_price']
# scrapy datetime
data['scrapy_datetime'] = datetime.today().strftime( '%Y-%m-%d %H:%M:%S' )


In [131]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url, headers=headers )

In [132]:
soup = BeautifulSoup( page.text, 'html.parser' )


In [133]:
total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get('data-total')
total_item


'84'

In [134]:
page_number = np.round( int( total_item ) / 36 )
page_number

2.0

In [135]:
url02 = url + '?page-size=' + str( int( page_number*36 ) )
url02


'https://www2.hm.com/en_us/men/products/jeans.html?page-size=72'

In [136]:
# API Requests
url = 'https://www2.hm.com/en_us/productpage.0636207010.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url, headers=headers )
# Beautiful Soup object
soup = BeautifulSoup( page.text, 'html.parser' )
# ==================== color name =================================
product_list = soup.find_all( 'a', class_='filter-option miniature' )
color_name = [p.get( 'data-color' ) for p in product_list]
# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]
df_color = pd.DataFrame( [product_id, color_name] ).T
df_color.columns = ['product_id', 'color_name']

# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

# ==================== composition =================================
product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]
# reaname dataframe
df_composition = pd.DataFrame( product_composition ).T
df_composition.columns = df_composition.iloc[0]
# delete first row
df_composition = df_composition.iloc[1:].fillna( method='ffill' )
# generate style id + color id
df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:])
# merge data color + decomposition
data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit','Composition']], how='left', on='style_id' )

In [137]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# empty dataframe
df_details = pd.DataFrame()
# unique columns for all products
aux = []
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )
for i in range( len( data ) ):
    
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    page = requests.get( url, headers=headers )
    
    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    # ==================== color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature' )
    color_name = [p.get( 'data-color' ) for p in product_list]
    
    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
    df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )
    
    # ==================== composition =================================
    product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
    product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]
    
    # reaname dataframe
    df_composition = pd.DataFrame( product_composition ).T
    df_composition.columns = df_composition.iloc[0]
    
    # delete first row
    df_composition = df_composition.iloc[1:].fillna( method='ffill' )
    
    # garantee the same number of columns
    df_composition = pd.concat( [df_pattern, df_composition], axis=0 )
    
    # generate style id + color id
    df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3] )
    df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:] )
    aux = aux + df_composition.columns.tolist()
    
    # merge data color + decomposition
    data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit','Composition', 'Size', 'Product safety']], how='left', on='style_id' )
    
    # all details products
    df_details = pd.concat( [df_details, data_sku], axis=0 )
    
    # Join Showroom data + details 
    data['style_id'] = data['product_id'].apply( lambda x: x[:-3] )
    data['color_id'] = data['product_id'].apply( lambda x: x[-3:] )
    data_raw = pd.merge( data, df_details[['style_id', 'color_name', 'Fit','Composition', 'Size', 'Product safety']], how='left', on='style_id' )
    
data_raw    

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,Composition,Size,Product safety
0,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-09-15 15:00:33,0690449,022,Light denim blue/trashed,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
1,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-09-15 15:00:33,0690449,022,Light denim blue/trashed,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
2,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-09-15 15:00:33,0690449,022,Denim blue,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
3,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-09-15 15:00:33,0690449,022,Denim blue,Skinny fit,"Cotton 98%, Elastane 2%","The model is 184cm/6'0"" and wears a size 31/32",NaN
4,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2021-09-15 15:00:33,0690449,022,Black/washed,Skinny fit,Lining: Polyester 100%,"The model is 184cm/6'0"" and wears a size 31/32",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1413,0811993021,men_jeans_regular,Regular Jeans,$ 29.99,2021-09-15 15:00:33,0811993,021,Pale denim blue,Regular fit,"Cotton 98%, Elastane 2%",NaN,\t\t\t FOR CHILD’S SAFE...
1414,0811993021,men_jeans_regular,Regular Jeans,$ 29.99,2021-09-15 15:00:33,0811993,021,Blue,Regular fit,"Cotton 98%, Elastane 2%",NaN,\r
1415,0811993021,men_jeans_regular,Regular Jeans,$ 29.99,2021-09-15 15:00:33,0811993,021,Blue,Regular fit,"Cotton 98%, Elastane 2%",NaN,\t\t\t FOR CHILD’S SAFE...
1416,0811993021,men_jeans_regular,Regular Jeans,$ 29.99,2021-09-15 15:00:33,0811993,021,Dark blue,Regular fit,"Cotton 98%, Elastane 2%",NaN,\r


In [138]:
data = pd.read_csv( 'products_hm' )
# product id
#data = data.dropna( subset=['product_id'] )
data['product_id'] = data['product_id'].astype( int )
# product name
data['product_name'] = data['product_name'].apply( lambda x: x.replace( ' ','_' ).lower() )
# product price
data['product_price'] = data['product_price'].apply( lambda x: x.replace( '$ ','' ) ).astype( float )
# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime( data['scrapy_datetime'],format='%Y-%m-%d %H:%M:%S' )
# style id
data['style_id'] = data['style_id'].astype( int )
# color id
data['color_id'] = data['color_id'].astype( int )
# color name
data['color_name'] = data['color_name'].apply( lambda x: x.replace( ' ', '_' ).replace( '/', '_' ).lower() if pd.notnull( x ) else x )
# fit
data['Fit'] = data['Fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )
# size number
data['size_number'] = data['Size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x )
data['size_number'] = data['size_number'].apply( lambda x: re.search( '\d+', x).group(0) if pd.notnull( x ) else x )
# size model
data['size_model'] = data['Size'].str.extract( '(\d+/\\d+)' )
data = data.drop( columns=['Size', 'Product safety','Unnamed: 0'], axis=1 )
# composition
data = data[~data['Composition'].str.contains('Pocket lining:', na = False)]
data = data[~data['Composition'].str.contains('Lining:', na = False)]
data = data[~data['Composition'].str.contains('Shell:', na = False)]
data = data[~data['Composition'].str.contains('Pocket:', na = False)]
data = data.reset_index(drop=True)

#break composition by comma
df1 = data['Composition'].str.split(',', expand = True)


In [139]:
#cotton | polyester | elastane | elasterell
df_ref = pd.DataFrame(index= np.arange(len(data)), columns = ['cotton','polyester','elastane','elasterell'])
                      
#cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref, df_cotton], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

#polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na = True),1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

#elastane
df_elastane = df1.loc[df1[1].str.contains('Elastane', na = True),1]
df_elastane.name = 'elastane'

df_ref = pd.concat([df_ref, df_elastane], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

#elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na = True),1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

#final join
data = pd.concat([data, df_ref], axis = 1)

#format composition data
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['elastane'] = data['elastane'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['elasterell'] = data['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)

data

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,Composition,size_number,size_model,cotton,polyester,elastane,elasterell
0,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-14 23:22:03,690449,22,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Elastane 2%",184,31/32,0.98,NaN,0.02,NaN
1,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-14 23:22:03,690449,22,denim_blue,skinny_fit,"Cotton 98%, Elastane 2%",184,31/32,0.98,NaN,0.02,NaN
2,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-14 23:22:03,690449,22,black_washed,skinny_fit,"Cotton 98%, Elastane 2%",184,31/32,0.98,NaN,0.02,NaN
3,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-14 23:22:03,690449,22,light_denim_blue,skinny_fit,"Cotton 98%, Elastane 2%",184,31/32,0.98,NaN,0.02,NaN
4,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-14 23:22:03,690449,22,black_washed_out,skinny_fit,"Cotton 98%, Elastane 2%",184,31/32,0.98,NaN,0.02,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1004476005,men_jeans_slim,freefit®_slim_jeans,49.99,2021-09-14 23:22:03,1004476,5,dark_denim_blue,slim_fit,"Cotton 90%, Elasterell-P 8%, Elastane 2%",NaN,NaN,0.90,NaN,NaN,0.08
391,1004476005,men_jeans_slim,freefit®_slim_jeans,49.99,2021-09-14 23:22:03,1004476,5,black_no_fade_black,slim_fit,"Cotton 90%, Elasterell-P 8%, Elastane 2%",NaN,NaN,0.90,NaN,NaN,0.08
392,1004476005,men_jeans_slim,freefit®_slim_jeans,49.99,2021-09-14 23:22:03,1004476,5,light_denim_blue,slim_fit,"Cotton 90%, Elasterell-P 8%, Elastane 2%",182,31/32,0.90,NaN,NaN,0.08
393,1004476005,men_jeans_slim,freefit®_slim_jeans,49.99,2021-09-14 23:22:03,1004476,5,blue,slim_fit,"Cotton 90%, Elasterell-P 8%, Elastane 2%",182,31/32,0.90,NaN,NaN,0.08
